In [1]:
import re
import pandas as pd

publication_file = '../metadata/DBNL 20241212 Proza 19e en 20e eeuw met subgenres.xlsx'
pub_df = pd.read_excel(publication_file)

## Preprocessing

In [2]:
def extract_author_id(title_id):
    if title_id.startswith('_'):
        return None
    if m := re.match(r"^(\w+\d+)(\w+\d+)", title_id):
        author_id = m.group(1)
    else:
        print('unexpected title_id format:', title)
    return author_id


def extract_year(jaar):
    jaar = jaar.strip()
    year_start, year_end, precision = None, None, None
    if m := re.match(r"^\d{4}$", jaar):
        year_start = int(jaar)
        year_end = year_start
        precision = 'exact'
    elif m := re.match(r"^(\d{4})-(\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = int(m.group(2))
        precision = 'exact'
    elif m := re.match(r"^ca\. (\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = year_start
        precision = 'circa'
    elif m := re.match(r"^ca\. (\d{4})\?$", jaar):
        year_start = int(m.group(1))
        year_end = year_start
        precision = 'circa'
    elif m := re.match(r"^na (\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = year_start
        precision = 'after'
    elif m := re.match(r"^ca\. (\d{4})-(\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = int(m.group(2))
        precision = 'circa'
    else:
        print(f"#{jaar}#")
    return year_start, year_end, precision

pub_df['author_id'] = pub_df.ti_id.apply(extract_author_id)
pub_df.author_id
pub_df['year_start'], pub_df['year_end'], pub_df['year_precision'] = zip(*pub_df.jaar.apply(extract_year))
pub_df['decade_start'] = pub_df.year_start.apply(lambda x: int(x/10) * 10)
pub_df['decade_end'] = pub_df.year_end.apply(lambda x: int(x/10) * 10)

pub_df

,ti_id,titel,jaar,druk,categorie,voornaam,voorvoegsel,achternaam,uitgever,plaats_van_uitgave,...,genre,subgenre,geplaatst,link naar DBNL.org,author_id,year_start,year_end,year_precision,decade_start,decade_end
0,woen003lant04,De Lantaarn voor 1800,1800,1ste druk,werk,Pieter,van,Woensel,NaN,NaN,...,proza,tijdschrift / jaarboek,woen003lant04_01,https://www.dbnl.org/tekst/woen003lant04_01,woen003,1800,1800,exact,1800,1800
1,kist001leve01,"Het leven, gevoelens en zonderlinge reize van ...",1800,1ste druk,werk,Willem,NaN,Kist,NaN,Haarlem,...,proza,roman,kist001leve01_01,https://www.dbnl.org/tekst/kist001leve01_01,kist001,1800,1800,exact,1800,1800
2,woen003lant05,De Lantaarn voor 1801,1801,1ste druk,werk,Pieter,van,Woensel,NaN,NaN,...,proza,tijdschrift / jaarboek,woen003lant05_01,https://www.dbnl.org/tekst/woen003lant05_01,woen003,1801,1801,exact,1800,1800
3,simo001verh01,Verhandelingen,1802,1ste druk,werk,Arend,NaN,Fokke Simonsz,Johannes van der Heij,Amsterdam,...,proza,lezing / voordracht,simo001verh01_01,https://www.dbnl.org/tekst/simo001verh01_01,simo001,1802,1802,exact,1800,1800
4,simo001verh01,Verhandelingen,1802,1ste druk,werk,Arend,NaN,Fokke Simonsz,Johannes van der Heij,Amsterdam,...,proza,non-fictie/filosofie-ethiek,simo001verh01_01,https://www.dbnl.org/tekst/simo001verh01_01,simo001,1802,1802,exact,1800,1800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,slor001frik01,Fri-kontren-sma,ca. 1970,1ste druk,werk,Michaël,NaN,Slory,Z.n.,z.p.,...,proza,gedichten / dichtbundel,slor001frik01_01,https://www.dbnl.org/tekst/slor001frik01_01,slor001,1970,1970,circa,1970,1970
5724,aard002stij01,Stijfkopje omnibus (onder pseudoniem Ankie Aal...,ca. 1970,herdruk,herdruk,H.P.,van den,Aardweg,Geka,Alphen aan de Rijn,...,proza,novelle(n),aard002stij01_01,https://www.dbnl.org/tekst/aard002stij01_01,aard002,1970,1970,circa,1970,1970
5725,gras003daar01,"Daar ga je, Claudia!",ca. 1970-1980,?,werk,Cok,NaN,Grashoff,Het Goede Boek / Standaard,Huizen / Antwerpen,...,proza,roman,gras003daar01_01,https://www.dbnl.org/tekst/gras003daar01_01,gras003,1970,1980,circa,1970,1980
5726,nijn001tolh02,"Tolhuis ""Het zwarte paard""",na 1941,herdruk,herdruk,H.J.,van,Nijnatten-Doffegnies,Westfriesland,Hoorn,...,proza,roman,nijn001tolh02_01,https://www.dbnl.org/tekst/nijn001tolh02_01,nijn001,1941,1941,after,1940,1940


In [3]:
pub_df.genre.value_counts()

genre
proza    5728
Name: count, dtype: int64

In [4]:
s = pub_df.ti_id.value_counts()
s = s[s > 1]
pub_df[pub_df.ti_id.isin(s.index)].sort_values('ti_id')

,ti_id,titel,jaar,druk,categorie,voornaam,voorvoegsel,achternaam,uitgever,plaats_van_uitgave,...,genre,subgenre,geplaatst,link naar DBNL.org,author_id,year_start,year_end,year_precision,decade_start,decade_end
5493,_alp001alph01,Alphabet tot nut en vermaak der jeugd,ca. 1850-1860,4de druk,herdruk,Anoniem,NaN,Alphabet tot nut en vermaak der jeugd,H.T. Hendriksen,Rotterdam,...,proza,plaatwerk / prentenboek,_alp001alph01_01,https://www.dbnl.org/tekst/_alp001alph01_01,None,1850,1860,circa,1850,1860
5494,_alp001alph01,Alphabet tot nut en vermaak der jeugd,ca. 1850-1860,4de druk,herdruk,Anoniem,NaN,Alphabet tot nut en vermaak der jeugd,H.T. Hendriksen,Rotterdam,...,proza,leerdicht,_alp001alph01_01,https://www.dbnl.org/tekst/_alp001alph01_01,None,1850,1860,circa,1850,1860
1164,_alv001190001,Alvoorder. Jaargang 1,1900-1901,1ste druk,werk,NaN,NaN,[tijdschrift] Alvoorder,Z.n.,Antwerpen/Brussel,...,proza,verhalen,_alv001190001_01,https://www.dbnl.org/tekst/_alv001190001_01,None,1900,1901,exact,1900,1900
1163,_alv001190001,Alvoorder. Jaargang 1,1900-1901,1ste druk,werk,NaN,NaN,[tijdschrift] Alvoorder,Z.n.,Antwerpen/Brussel,...,proza,gedichten / dichtbundel,_alv001190001_01,https://www.dbnl.org/tekst/_alv001190001_01,None,1900,1901,exact,1900,1900
1162,_alv001190001,Alvoorder. Jaargang 1,1900-1901,1ste druk,werk,NaN,NaN,[tijdschrift] Alvoorder,Z.n.,Antwerpen/Brussel,...,proza,tijdschrift / jaarboek,_alv001190001_01,https://www.dbnl.org/tekst/_alv001190001_01,None,1900,1901,exact,1900,1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4477,zout002gesc01,Het geschenk van Watramama,1980,1ste druk,werk,H.,NaN,Zoutendijk,Stichting Kinderkrant Suriname,Paramaribo,...,proza,verhalen,zout002gesc01_01,https://www.dbnl.org/tekst/zout002gesc01_01,zout002,1980,1980,exact,1980,1980
2965,zuyl002gesc01,De geschiedenis van Caliste,1942,1ste druk,uitgave,Belle,van,Zuylen,Veen,Amsterdam,...,proza,briefroman,zuyl002gesc01_01,https://www.dbnl.org/tekst/zuyl002gesc01_01,zuyl002,1942,1942,exact,1940,1940
2966,zuyl002gesc01,De geschiedenis van Caliste,1942,1ste druk,uitgave,Belle,van,Zuylen,Veen,Amsterdam,...,proza,vertaling,zuyl002gesc01_01,https://www.dbnl.org/tekst/zuyl002gesc01_01,zuyl002,1942,1942,exact,1940,1940
4326,zuyl002rebe01,Rebels en beminnelijk,1978,3de druk,herdruk,Belle,van,Zuylen,De Arbeiderspers,Amsterdam,...,proza,vertaling,zuyl002rebe01_01,https://www.dbnl.org/tekst/zuyl002rebe01_01,zuyl002,1978,1978,exact,1970,1970


In [5]:
s = pub_df.subgenre.value_counts()
list(s[s > 10].index)

['roman',
 'tijdschrift / jaarboek',
 'verhalen',
 'novelle(n)',
 'verzameld werk',
 'historische roman',
 'gedichten / dichtbundel',
 'plaatwerk / prentenboek',
 'non-fictie/essays-opstellen',
 'schetsen',
 'bloemlezing',
 'reisbeschrijving',
 'non-fictie/autobiografie-memoires',
 'kroniek',
 'kritiek(en)',
 'detective',
 'sprookje(s)',
 'non-fictie/brieven',
 'reisverhalen',
 'column(s) / cursiefjes',
 'non-fictie/theologie',
 'briefroman',
 'roman (populair/bestseller)',
 'dierenverhaal/-epos',
 'non-fictie/biografie',
 'non-fictie/dagboek',
 'aforismen',
 'non-fictie/interview(s)',
 'traktaat',
 'satire',
 'vertaling',
 'studie',
 'legende-mythe-sage',
 'non-fictie/koloniën-reizen',
 'marialegende',
 'non-fictie/schoolboek',
 'schelmenverhaal',
 'lezing / voordracht',
 'vertaling: Frans / Nederlands',
 'limburg',
 'non-fictie/geschiedenis-archeologie',
 'bijbel / bijbeltekst(en)',
 'vertaling: Amerikaans-Engels / Nederlands',
 'proefschrift',
 'heiligenleven',
 'non-fictie/kunstges

In [6]:
prose_fiction_genres = [
    'roman',
    'novelle(n)',
    'verhalen',
    'historische roman',
    'detective',
    'briefroman',
    'roman (populair/bestseller)',
    # 'dierenverhaal/-epos',
    # 'aforismen',
    # 'satire',
    # 'vertaling',
    # 'legende-mythe-sage',
    # 'marialegende',
    'schelmenverhaal',
    # 'vertaling: Frans / Nederlands',
    # 'bijbel / bijbeltekst(en)',
    # 'vertaling: Amerikaans-Engels / Nederlands',
    # 'heiligenleven',
    # 'toneeltekst (modern)',
    'ridderroman',
]

## Removing anonymous titles and magazines

In [7]:
# if `Voornaam` is `Anoniem`, the author is anonymous
pub_df['is_anonymous'] = pub_df.voornaam.apply(lambda x: True if x == 'Anoniem' else False)
print(f"number of titles with anonymous authors: {len(pub_df[pub_df.is_anonymous == True])}")

# if the `ti_id` starts with an underscore, there is no seperate author ID
pub_df['no_author_id'] = pub_df.ti_id.str.startswith('_')
print(f"number of titles with no author ID: {len(pub_df[pub_df.no_author_id == True])}")

# if the `ti_id` starts with an underscore, there is no seperate author ID
pub_df['is_magazine'] = pub_df.subgenre.str.contains('tijdschrift')
print(f"number of titles that are magazines: {len(pub_df[pub_df.is_magazine == True])}")



number of titles with anonymous authors: 182
number of titles with no author ID: 1514
number of titles that are magazines: 1290


In [8]:
anonymous_ti_ids = list(pub_df[pub_df.is_anonymous == True].ti_id.unique())
authorless_ti_ids = list(pub_df[pub_df.no_author_id == True].ti_id.unique())
magazine_ti_ids = list(pub_df[pub_df.is_magazine == True].ti_id.unique())

remove_ids = list(set(anonymous_ti_ids + magazine_ti_ids + authorless_ti_ids))
print('number of title ids to remove:', len(remove_ids))
pub_df = pub_df[pub_df.ti_id.isin(remove_ids) == False]
pub_df.shape

number of title ids to remove: 1378


(4146, 24)

In [9]:
pub_df = pub_df[(pub_df.is_anonymous == False) & (pub_df.no_author_id == False)]
s = pub_df.subgenre.value_counts()
s[s > 10]

# removing multi-author magazines/periodicals
pub_df = pub_df[pub_df.subgenre.apply(lambda x: False if 'tijdschrift' in x else True)]

In [10]:
pub_df.shape

(4146, 24)

## Extra check: are there multiple print editions of a work?

In [11]:
pub_df[['ti_id', 'druk']].drop_duplicates().ti_id.value_counts()



ti_id
kist001leve01    1
butn001nood01    1
eeml001scha02    1
last001spaa02    1
kool009scho01    1
                ..
holt075scho01    1
holt075broe01    1
reyn008insh01    1
reyn008mani01    1
bosm029toch02    1
Name: count, Length: 3168, dtype: int64

No, although for some titles we only have the second, third, or later print edition, for each work we only have one edition.

## Removing titles with multiple authors

In [12]:

ti_auth = pub_df[['ti_id', 'voornaam', 'voorvoegsel', 'achternaam']].drop_duplicates()
num_authors = ti_auth.ti_id.value_counts()
multi_authors = num_authors[num_authors > 1]
pub_df[pub_df.ti_id.isin(multi_authors.index)]

,ti_id,titel,jaar,druk,categorie,voornaam,voorvoegsel,achternaam,uitgever,plaats_van_uitgave,...,link naar DBNL.org,author_id,year_start,year_end,year_precision,decade_start,decade_end,is_anonymous,no_author_id,is_magazine
5,wolf016gesc01,Geschrift eener bejaarde vrouw,1802,1ste druk,werk,Betje,NaN,Wolff,Isaac van Cleef,Den Hage,...,https://www.dbnl.org/tekst/wolf016gesc01_01,wolf016,1802,1802,exact,1800,1800,False,False,False
6,wolf016gesc01,Geschrift eener bejaarde vrouw,1802,1ste druk,werk,Aagje,NaN,Deken,Isaac van Cleef,Den Hage,...,https://www.dbnl.org/tekst/wolf016gesc01_01,wolf016,1802,1802,exact,1800,1800,False,False,False
146,blom013iwei01,Iwein van Aelst (1128); Het blijspel van Mevro...,1842,1ste druk,werk,A.L.G.,NaN,Bosboom-Toussaint,NaN,NaN,...,https://www.dbnl.org/tekst/blom013iwei01_01,blom013,1842,1842,exact,1840,1840,False,False,False
148,blom013iwei01,Iwein van Aelst (1128); Het blijspel van Mevro...,1842,1ste druk,werk,Ph.,NaN,Blommaert,NaN,NaN,...,https://www.dbnl.org/tekst/blom013iwei01_01,blom013,1842,1842,exact,1840,1840,False,False,False
227,zegg002inde01,In de speeluren,1850,1ste druk,werk,A.L.H.,NaN,Ising,D.J. Couvee,Leyden,...,https://www.dbnl.org/tekst/zegg002inde01_01,zegg002,1850,1850,exact,1850,1850,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,tien010scha01,De schatten van Jantje,ca. 1943,2de druk,herdruk,A.B.,van,Tienhoven,Gebr. Kluitman,Alkmaar,...,https://www.dbnl.org/tekst/tien010scha01_01,tien010,1943,1943,circa,1940,1940,False,False,False
5687,tien010scha01,De schatten van Jantje,ca. 1943,2de druk,herdruk,A.,van,Tienhoven,Gebr. Kluitman,Alkmaar,...,https://www.dbnl.org/tekst/tien010scha01_01,tien010,1943,1943,circa,1940,1940,False,False,False
5688,tien010scha01,De schatten van Jantje,ca. 1943,2de druk,herdruk,A.,van,Tienhoven,Gebr. Kluitman,Alkmaar,...,https://www.dbnl.org/tekst/tien010scha01_01,tien010,1943,1943,circa,1940,1940,False,False,False
5713,groo050ondr01,Ondrofeni sa leri ju: Tori's,ca. 1950,1ste druk,werk,Antoon,NaN,Donicie,Varekamp & Co,Paramaribo,...,https://www.dbnl.org/tekst/groo050ondr01_01,groo050,1950,1950,circa,1950,1950,False,False,False


In [13]:
pub_df = pub_df[pub_df.ti_id.isin(multi_authors.index) == False]
pub_df.shape

(3854, 24)

## Filtering prose fiction genres

In [14]:
non_prose_ids = pub_df[pub_df.subgenre.isin(prose_fiction_genres) == False].ti_id.unique()

In [15]:
non_prose_ids.shape

(1010,)

### Collapsing subgenre labels

Titles with multiple subgenres (where all subgenres are in our `prose_fiction_genres` list) have one row per subgenre. We want to merge the subgenres.

In [16]:
pub_df['is_prose'] = pub_df.ti_id.apply(lambda x: False if x in non_prose_ids else True)
pub_df.is_prose.value_counts()

is_prose
True     2154
False    1700
Name: count, dtype: int64

In [17]:
#pub_df['is_prose'] = pub_df.subgenre.apply(lambda x: True if x in prose_fiction_genres else False)
#non_prose_ids = pub_df[pub_df.is_prose == False].ti_id.unique()
prose_df = pub_df[pub_df.is_prose]

#prose_df = pub_df[pub_df.subgenre.isin(prose_fiction_genres)]
pub_df.shape, prose_df.shape


((3854, 25), (2154, 25))

In [18]:
prose_df = prose_df.drop('subgenre', axis=1).drop_duplicates()
prose_df.shape

(2069, 24)

Make sure that title IDs only occur once:

In [19]:
prose_df.ti_id.value_counts()

ti_id
kist001leve01    1
teir001mijn01    1
liuw002jell01    1
nijh002venu01    1
noor022kanm01    1
                ..
holt075scho01    1
gogh002hoog01    1
eman001mens01    1
coen011eila02    1
bosm029toch02    1
Name: count, Length: 2069, dtype: int64

In [20]:
pub_df = pub_df.drop('subgenre', axis=1).drop_duplicates()
pub_df.to_csv('../metadata/DBNL-single_author_pub_titles.tsv', sep='\t', index=False)

In [21]:
prose_df.to_csv('../metadata/DBNL-single_author_prose_titles.tsv', sep='\t', index=False)